Generate llama3.3 result

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

question_df = pd.read_json('eval/questions.json', lines=True)

def time_handler(time_interval:str):
    if time_interval=="Now":
        return datetime.strptime("2025/1/21 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few hours before":
        return datetime.strptime("2025/1/20 08:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few days before":
        return datetime.strptime("2025/1/15 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few weeks before":
        return datetime.strptime("2024/12/16 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few months before":
        return datetime.strptime("2024/9/22 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A couple of years before":
        return datetime.strptime("2022/10/06 12:00", "%Y/%m/%d %H:%M")

for row in range(len(question_df)):
    for j, col in enumerate(['first_session_dialogue', 'second_session_dialogue', 'third_session_dialogue', 'fourth_session_dialogue', 'fifth_session_dialogue']):
        dialog = []
        current_time = time_handler(question_df['time_interval'][row][j])
        for i in range(int(len(question_df[col][row])/2)):
            current_time = current_time + timedelta(minutes=1)
            log = {
                "user":question_df[col][row][i*2],
                "assistant":question_df[col][row][i*2+1],
                "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")
            }
            dialog.append(log)
        question_df.loc[row, col] = str(dialog)
question_df = question_df.drop(columns=['time_interval', 'double_check', 'reason', 'relationship'], axis=1)
question_df

In [2]:
from long_memory.component import WeaviateLongMemory

long_mem = WeaviateLongMemory(user='llama', model='llama3.3')

Detect empty group memory, create memory space...
Detect empty child memory, create memory space...


In [5]:
index = ['long_mem_result', 'long_mem_recall_result']
for i in index:
    question_df[i]=None

In [13]:
for row in range(23, 100):
    print(f'==={row}===')
    
    question = f"user:{eval(question_df['question'][row])['user']}"
            
    # long mem process
    success = False
    while not success:
        try:
            long_mem.del_memory()
            for col in ['first_session_dialogue', 'second_session_dialogue', 'third_session_dialogue', 'fourth_session_dialogue', 'fifth_session_dialogue']:
                conv = eval(question_df[col][row])
                chat_logs = []
                for log in conv:
                    chat_logs.append({
                        "text":f"user:{log['user']}, assistant:{log['assistant']}",
                        "time":log['time']
                    })
                long_mem.add_chat_logs(chat_logs)
            searched_memory = long_mem.get_memory(question, recall=False)
            question_df.loc[row, 'long_mem_result'] = str(searched_memory)
            # (recall)
            searched_memory = long_mem.get_memory(question, recall=True)
            question_df.loc[row, 'long_mem_recall_result'] = str(searched_memory)
            success = True
        except Exception as e:
            print(f"---long memory error:{e}, retry...---")
    
    #if (row+1)%10==0:
    question_df.to_json("llama_result.json", orient="records", lines=True)

===19===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
===20===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
===21===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
===22===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long m

KeyboardInterrupt: 

Score

In [59]:
question_df['long_score'] = None
question_df['long_recall_score'] = None

origin_df = pd.read_json('eval/eval_result_11_29.json', lines=True)

In [61]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import re

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content
    
def llm_response_handler(response:str):
    """handle llm response format, especially for llama family"""
    try:
        return json.loads(response)
    except:
        response = response.strip()
        try:
            return json.loads(re.search(r"```json(.*?)```", response, re.DOTALL).group(1).strip())
        except:
            try:
                return json.loads(re.search(r"```(.*?)```", response, re.DOTALL).group(1).strip())
            except:
                return response

In [159]:
evaluation_prompt = """You will act as a reviewer to score the following returned memory based on their relevance, conciseness and readability.
The fewer and more relevant the key messages are, the higher the score will be.
Here is the question and answer:{qa_pair}

The score range is from 1 to 10, with 1 being the worst and 10 being the best. Please refer to the following three examples for guidance:
A_system:{a_system}, score:{a_score}
B_system:{b_system}, score:{b_score}
C_system:{c_system}, score:{c_score}

Next, please score the returned memory for the following systems:
Long_memory_system:{long_memory_system}

Long_recall_memory_system:{long_recall_memory_system}

Please provide the scores in the following format:
```json
{{
    "long_memory_system":"1~10",
    "long_recall_memory_system":"1~10"
}}
```"""

In [168]:
for row in range(500):
    print(f'==={row}===')
    
    qa_pair = question_df['question'][row]
    
    short_searched = origin_df.loc[row, 'short_mem_result']
    short_score = origin_df.loc[row, 'short_score']
    base_dialog_searched = origin_df.loc[row, 'base_dialog']
    base_dialog_score = origin_df.loc[row, 'base_dialog_score']
    base_paragraph_searched = origin_df.loc[row, 'base_paragraph']
    base_paragraph_score = origin_df.loc[row, 'base_paragraph_score']
    
    long_searched = question_df.loc[row, 'long_mem_result']
    long_recall_searched = question_df.loc[row, 'long_mem_recall_result']
    
    p = evaluation_prompt.format(qa_pair=qa_pair, a_system=short_searched, a_score=short_score, b_system=base_dialog_searched, b_score=base_dialog_score, c_system=base_paragraph_searched, c_score=base_paragraph_score, long_memory_system=long_searched, long_recall_memory_system=long_recall_searched)
    res = llm_create(p)
    res_dict = llm_response_handler(res)
    
    question_df.loc[row, 'long_score'] = res_dict.get('long_memory_system')
    question_df.loc[row, 'long_recall_score'] = res_dict.get('long_recall_memory_system')
    # print(res_dict.get('reason'))
    
    if row%50==0:
        question_df.to_json("eval_result_llama.json", orient="records", lines=True)

===200===
===201===
===202===
===203===
===204===
===205===
===206===
===207===
===208===
===209===
===210===
===211===
===212===
===213===
===214===
===215===
===216===
===217===
===218===
===219===
===220===
===221===
===222===
===223===
===224===
===225===
===226===
===227===
===228===
===229===
===230===
===231===
===232===
===233===
===234===
===235===
===236===
===237===
===238===
===239===
===240===
===241===
===242===
===243===
===244===
===245===
===246===
===247===
===248===
===249===
===250===
===251===
===252===
===253===
===254===
===255===
===256===
===257===
===258===
===259===
===260===
===261===
===262===
===263===
===264===
===265===
===266===
===267===
===268===
===269===
===270===
===271===
===272===
===273===
===274===
===275===
===276===
===277===
===278===
===279===
===280===
===281===
===282===
===283===
===284===
===285===
===286===
===287===
===288===
===289===
===290===
===291===
===292===
===293===
===294===
===295===
===296===
===297===
===298===
===299===


In [6]:
number=500
observe_df = question_df.drop(['first_session_dialogue', 'second_session_dialogue', 'third_session_dialogue', 'fourth_session_dialogue', 'fifth_session_dialogue'], axis=1)[:number]
print(observe_df['long_score'].astype(int).sum()/number)
print(observe_df['long_recall_score'].astype(int).sum()/number)

8.126
8.23


In [ ]:
index=1
print(observe_df['long_recall_score'][index])
eval(observe_df['long_mem_recall_result'][index])

In [ ]:
print(observe_df['long_score'][index])
eval(observe_df['long_mem_result'][index])

In [167]:
observe_df['question'][index]

"{'user': 'Do you remember when we were talking about the beautiful house you got? What did I say was the least I could do?', 'assistant': 'You said buying me the house was the least you could do, and you were just happy to see me happy.'}"